In [1]:
import numpy as np
import sklearn.metrics
from keras import backend as K
import tensorflow as tf

Using TensorFlow backend.


In [22]:
y_true = np.array([[1, 0, 0, 0],
                   [0, 1, 1, 0],
                  [1, 0, 1, 0],
                  [0, 1, 0, 1]])
y_pred = np.array([[0.5, 0.0, 0.0, 0.0],
                   [0.55, 0.3, 0.05, 0.1],
                  [0.8, 0.9, 0.1, 0.2],
                  [0.55, 0.3, 0.05, 0.1]])

tf_y_true = tf.constant(y_true)
tf_y_pred = tf.constant(y_pred)

truth = K.placeholder(shape=[None, 4])
score = K.placeholder(shape=[None, 4])

In [23]:
# from official code https://colab.research.google.com/drive/1AgPdhSp7ttY18O3fEoHOQKlt_3HJDLi8#scrollTo=cRCaCIb9oguU
def _one_sample_positive_class_precisions(scores, truth):
    """Calculate precisions for each true class for a single sample.

    Args:
      scores: np.array of (num_classes,) giving the individual classifier scores.
      truth: np.array of (num_classes,) bools indicating which classes are true.

    Returns:
      pos_class_indices: np.array of indices of the true classes for this sample.
      pos_class_precisions: np.array of precisions corresponding to each of those
        classes.
    """
    num_classes = scores.shape[0]
    print("num_classes:", num_classes)
    pos_class_indices = np.flatnonzero(truth > 0)
    print("pos_class_indices:",pos_class_indices)
    # Only calculate precisions if there are some true classes.
    if not len(pos_class_indices):
        return pos_class_indices, np.zeros(0)
    # Retrieval list of classes for this sample.
    retrieved_classes = np.argsort(scores)[::-1]
    print("retrieved_classes:", retrieved_classes)
    # class_rankings[top_scoring_class_index] == 0 etc.
    class_rankings = np.zeros(num_classes, dtype=np.int)
    print('class_rankings:', class_rankings)
    class_rankings[retrieved_classes] = range(num_classes)
    print('class_rankings:', class_rankings)
    # Which of these is a true label?
    retrieved_class_true = np.zeros(num_classes, dtype=np.bool)
    print('retrieved_class_true:', retrieved_class_true)
    retrieved_class_true[class_rankings[pos_class_indices]] = True
    print('retrieved_class_true:', retrieved_class_true)
    # Num hits for every truncated retrieval list.
    retrieved_cumulative_hits = np.cumsum(retrieved_class_true)
    print('retrieved_cumulative_hits:', retrieved_cumulative_hits)
    # Precision of retrieval list truncated at each hit, in order of pos_labels.
    precision_at_hits = (
            retrieved_cumulative_hits[class_rankings[pos_class_indices]] /
            (1 + class_rankings[pos_class_indices].astype(np.float)))
    return pos_class_indices, precision_at_hits


def calculate_per_class_lwlrap(truth, scores):
    """Calculate label-weighted label-ranking average precision.

    Arguments:
      truth: np.array of (num_samples, num_classes) giving boolean ground-truth
        of presence of that class in that sample.
      scores: np.array of (num_samples, num_classes) giving the classifier-under-
        test's real-valued score for each class for each sample.

    Returns:
      per_class_lwlrap: np.array of (num_classes,) giving the lwlrap for each
        class.
      weight_per_class: np.array of (num_classes,) giving the prior of each
        class within the truth labels.  Then the overall unbalanced lwlrap is
        simply np.sum(per_class_lwlrap * weight_per_class)
    """
    assert truth.shape == scores.shape
    num_samples, num_classes = scores.shape
    # Space to store a distinct precision value for each class on each sample.
    # Only the classes that are true for each sample will be filled in.
    precisions_for_samples_by_classes = np.zeros((num_samples, num_classes))
    for sample_num in range(num_samples):
        pos_class_indices, precision_at_hits = (
            _one_sample_positive_class_precisions(scores[sample_num, :],
                                                  truth[sample_num, :]))
        print("pos_class_indices, precision_at_hits:", pos_class_indices, precision_at_hits)
        precisions_for_samples_by_classes[sample_num, pos_class_indices] = (
            precision_at_hits)
    labels_per_class = np.sum(truth > 0, axis=0)
    weight_per_class = labels_per_class / float(np.sum(labels_per_class))
    # Form average of each column, i.e. all the precisions assigned to labels in
    # a particular class.
    per_class_lwlrap = (np.sum(precisions_for_samples_by_classes, axis=0) /
                        np.maximum(1, labels_per_class))
    # overall_lwlrap = simple average of all the actual per-class, per-sample precisions
    #                = np.sum(precisions_for_samples_by_classes) / np.sum(precisions_for_samples_by_classes > 0)
    #           also = weighted mean of per-class lwlraps, weighted by class label prior across samples
    #                = np.sum(per_class_lwlrap * weight_per_class)
    return sum(per_class_lwlrap*weight_per_class), per_class_lwlrap, weight_per_class 

In [24]:
calculate_per_class_lwlrap(y_true, y_pred)

num_classes: 4
pos_class_indices: [0]
retrieved_classes: [0 3 2 1]
class_rankings: [0 0 0 0]
class_rankings: [0 3 2 1]
retrieved_class_true: [False False False False]
retrieved_class_true: [ True False False False]
retrieved_cumulative_hits: [1 1 1 1]
pos_class_indices, precision_at_hits: [0] [1.]
num_classes: 4
pos_class_indices: [1 2]
retrieved_classes: [0 1 3 2]
class_rankings: [0 0 0 0]
class_rankings: [0 1 3 2]
retrieved_class_true: [False False False False]
retrieved_class_true: [False  True False  True]
retrieved_cumulative_hits: [0 1 1 2]
pos_class_indices, precision_at_hits: [1 2] [0.5 0.5]
num_classes: 4
pos_class_indices: [0 2]
retrieved_classes: [1 0 3 2]
class_rankings: [0 0 0 0]
class_rankings: [1 0 3 2]
retrieved_class_true: [False False False False]
retrieved_class_true: [False  True False  True]
retrieved_cumulative_hits: [0 1 1 2]
pos_class_indices, precision_at_hits: [0 2] [0.5 0.5]
num_classes: 4
pos_class_indices: [1 3]
retrieved_classes: [0 1 3 2]
class_rankings: 

(0.5952380952380951,
 array([0.75      , 0.5       , 0.5       , 0.66666667]),
 array([0.28571429, 0.28571429, 0.28571429, 0.14285714]))

In [8]:
def tf_one_sample_positive_class_precisions(y_true, y_pred) :
    num_samples, num_classes = y_pred.shape
    
    # find true labels
    pos_class_indices = tf.where(y_true > 0) 
    
    # put rank on each element
    class_rankings = tf.nn.top_k(y_pred, k=num_classes).indices 
    
    #pick_up ranks
    num_correct_until_correct = tf.gather_nd(class_rankings, pos_class_indices) 
    
    # add one for division for "presicion_at_hits"
    num_correct_until_correct_one = tf.add(num_correct_until_correct, 1) 
    num_correct_until_correct_one = tf.cast(num_correct_until_correct_one, tf.float32)
    
    # generate tensor [num_sample, predict_rank], 
    # top-N predicted elements have flag, N is the number of positive for each sample.
    sample_label = pos_class_indices[:, 0]   
    sample_label = tf.reshape(sample_label, (-1, 1))
    sample_label = tf.cast(sample_label, tf.int32)
    num_correct_until_correct = tf.reshape(num_correct_until_correct, (-1, 1))
    
    retrieved_class_true_position = tf.concat((sample_label, 
                                               num_correct_until_correct), axis=1)
    retrieved_pos = tf.ones(shape=tf.shape(retrieved_class_true_position)[0], dtype=tf.int32)
    retrieved_class_true = tf.scatter_nd(retrieved_class_true_position, 
                                         retrieved_pos, 
                                         tf.shape(y_pred))
    print(K.eval(retrieved_class_true_position))
    print(K.eval(retrieved_pos))
    print(K.eval(retrieved_class_true))
    
    # cumulate predict_rank
    retrieved_cumulative_hits = tf.cumsum(retrieved_class_true, axis=1)

    # find positive position
    pos_ret_indices = tf.where(retrieved_class_true > 0)

    # find cumulative hits
    correct_rank = tf.gather_nd(retrieved_cumulative_hits, pos_ret_indices)  
    correct_rank = tf.cast(correct_rank, tf.float32)

    # compute presicion
    precision_at_hits = tf.truediv(correct_rank, num_correct_until_correct_one)
 
    return pos_class_indices, precision_at_hits

def tf_lwlrap(y_true, y_pred):
    num_samples, num_classes = y_pred.shape
    
    pos_class_indices, precision_at_hits = (tf_one_sample_positive_class_precisions(y_true, y_pred))
    
    pos_flgs = tf.cast(y_true > 0, tf.int32)
    
    labels_per_class = tf.reduce_sum(pos_flgs, axis=0)
    
    weight_per_class = tf.truediv(tf.cast(labels_per_class, tf.float32),
                                  tf.cast(tf.reduce_sum(labels_per_class), tf.float32))
    
    sum_precisions_by_classes = tf.zeros(shape=(num_classes), dtype=tf.float32)  
    
    class_label = pos_class_indices[:,1]

    sum_precisions_by_classes = tf.unsorted_segment_sum(precision_at_hits,
                                                        class_label,
                                                       num_classes)
    
    labels_per_class = tf.cast(labels_per_class, tf.float32)
    labels_per_class = tf.add(labels_per_class, 1e-7)
    per_class_lwlrap = tf.truediv(sum_precisions_by_classes,
                                  tf.cast(labels_per_class, tf.float32))
    
    out = tf.cast(tf.tensordot(per_class_lwlrap, weight_per_class, axes=1), dtype=tf.float32)
    return out

In [9]:
x = tf_lwlrap(tf_y_true, tf_y_pred)
sess = tf.Session()
sess.run(x)

InternalError: Failed to create session.

In [ ]:
x = tf_lwlrap(truth, score)
sess = tf.Session()
sess.run(x,  feed_dict={truth:y_true, score:y_pred})